Hello,

In this work, I will try to recognize emotions in photographs through a convolutional neural network using a keras library.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras import models
from keras import layers
from keras import optimizers
from keras.utils import to_categorical

**DATA**


Over 35,000 photographs collected in the data in 2013, some of the existing photographs were taken from various media publications and some of them consist of stock photographs. Our goal here is to recognize 7 different emotions (angry, happy, scared, natural, etc.) through photographs with a convolutional neural network.

[](http://)**Determine a Road Map**

First, we will load our data and examine the various information in it.

Later, we will make our train and test data to be able to model with convolutional neural network and build a model and expect to get a good result.

**UPLOAD DATA**

In [ ]:
data = pd.read_csv("/kaggle/input/facial-expression-recognitionferchallenge/fer2013/fer2013/fer2013.csv")
data.head()

**FIRST LOOK**

In [ ]:
print("Datanın satır ve sütün sayıları = ", data.shape)
print("Sütünların ismi = ", data.columns)

The emotion variable represents emotions and is our target variable.
Pixels variable expresses the value per pixel in the photos.
Usage, on the other hand, shows which set the row it belongs to (such as training and testing).

Let's look at our training and test sets and separate these sets.

In [ ]:
data["Usage"].value_counts()

In [ ]:
training = data.loc[data["Usage"] == "Training"]
public_test = data.loc[data["Usage"] == "PublicTest"]
private_test = data.loc[data["Usage"] == "PrivateTest"]

print("Eğitim seti = ", training.shape)
print("Genel test seti = ", public_test.shape)
print("Özel test seti = ", private_test.shape)

Her bir veri setimizde hangi duygudan ne kadar olduğunu görmek istersek;

In [ ]:
print("========================= Emotion Adetleri ===========================")
print("train adet = \n{}, \npublic adet = \n{}, \nprivate adet = \n{}".format(training["emotion"].value_counts(),
      public_test["emotion"].value_counts(), private_test["emotion"].value_counts()))


Yine her bir setimizde "emotion", "pixels" ve "usage" değişkenlerimiz var şimdi eğitim ve test setlerimizi düzenleme işlemlerine geçelim.

**PREPROCESSING**

We will do the same for each set. These:

    1) We will categorize it by taking the "emotion" column which is our answers. In other words, we will output a transaction with 0 in all the remaining lines except the class it is in.
    
    2) We will take the pixels column in our set and turn it into a tensor and standardize it.
     

In [ ]:
train_labels = training["emotion"]
train_labels = to_categorical(train_labels)

train_pixels = training["pixels"].str.split(" ").tolist()
train_pixels = np.uint8(train_pixels)
train_pixels = train_pixels.reshape((28709, 48, 48, 1))
train_pixels = train_pixels.astype("float32") / 255


private_labels = private_test["emotion"]
private_labels = to_categorical(private_labels)

private_pixels = private_test["pixels"].str.split(" ").tolist()
private_pixels = np.uint8(private_pixels)
private_pixels = private_pixels.reshape((3589, 48, 48, 1))
private_pixels = private_pixels.astype("float32") / 255


public_labels = public_test["emotion"]
public_labels = to_categorical(public_labels)

public_pixels = public_test["pixels"].str.split(" ").tolist()
public_pixels = np.uint8(public_pixels)
public_pixels = public_pixels.reshape((3589, 48, 48, 1))
public_pixels = public_pixels.astype("float32") / 255

Now let's look at some photos

In [ ]:
import seaborn as sns
plt.figure(0, figsize=(12,6))
for i in range(1, 13):
    plt.subplot(3,4,i)
    plt.imshow(train_pixels[i, :, :, 0], cmap="gray")

plt.tight_layout()
plt.show()


Graf --> [https://www.kaggle.com/omarensaj/fer-emotion-detection-psd07](http://)

So far, we have processed the data and made it ready to build a model, now we can start building the model.

**MODEL**

In [ ]:
model_1 = models.Sequential()

# Conv (evrişim katmanı)
model_1.add(layers.Conv2D(64, (5, 5), activation='relu', input_shape=(48,48,1)))
#Ortaklama katmanı
model_1.add(layers.MaxPooling2D(pool_size=(5,5), strides=(2, 2)))

model_1.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_1.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_1.add(layers.AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

model_1.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_1.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_1.add(layers.AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

model_1.add(layers.Flatten())

# Tam bağlantı katmanı
model_1.add(layers.Dense(1024, activation='relu'))
model_1.add(layers.Dropout(0.2))
model_1.add(layers.Dense(1024, activation='relu'))
model_1.add(layers.Dropout(0.2))

model_1.add(layers.Dense(7, activation='softmax'))


In [ ]:
model_1.summary()

The number of transactions in the model output is almost 1.5 million. Let's train the model now

In [ ]:
model_1.compile(optimizer = "Adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

hist = model_1.fit(train_pixels, train_labels, batch_size = 256, epochs = 30,
                validation_data = (private_pixels, private_labels))


In [ ]:
acc = hist.history["accuracy"]
val_acc = hist.history["val_accuracy"]
loss = hist.history["loss"]
val_loss = hist.history["val_loss"]

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, "bo", label = "Eğitim Başarısı")
plt.plot(epochs, val_acc, "b", label = "Doğrulama Başarısı")
plt.title("Eğitim ve Doğrulama Başarısı")
plt.legend()

plt.figure()

plt.plot(epochs, loss, "bo", label = "Eğitim Kaybı")
plt.plot(epochs, val_loss, "b", label = "Doğrulama Kaybı")
plt.legend()


plt.show()

In [ ]:
print("Teşekkürler!")


Model 16-17. After the epoch (1 full round on the data), the training success increases and the test success decreases, that is, it overfit.

In this study, we were able to predict 7 different emotions by 58-60%. Data Diversification can be applied to photos in different filter sizes, more layers or training set to improve the model.

See you...

Thanks.